In [1]:
import numpy as np

def representation_polynomial(m,n):
    matroid = matroids.Uniform(m,n) # type: ignore
    lattice_of_flats = matroid.lattice_of_flats()
    maxChains = [
        [list(chain) for chain in p][1:] for p in list(lattice_of_flats.chains()) if len(p) == matroid.rank() + 1
    ]
    weights = generate_weights(matroid.rank())
    fy_monomials = generate_fyMonomials(maxChains, weights)
    conjugacy_Classes, class_actions = generate_conjugacyClasses(n)
    characters_list_graded = characters_list(fy_monomials, class_actions)
    representation_polynomial = decomposition(characters_list_graded, n)
    
    return representation_polynomial

def generate_weights(rank):
    weights = set() 
    for i in range(1, rank):
        for j in range(rank):
            weight = [0] * rank
            
            if i >= j:
                weight[i] = j
                weights.add(tuple(weight))

            if rank - (i+1) > 1:
                y = generate_weights(rank - (i + 1))  # Recursion
                for x in y:
                    temp_weight = weight.copy()
                    weights.add(tuple(temp_weight[:i+1] + x)) 

    return [list(w) for w in weights]

def generate_fyMonomials(maxChains, weights):
    fy_monomials = [set() for _ in range(len(weights[0]))]
    for chain in maxChains:
        for weight in weights:
            degree = sum(weight)
            fy_monomial = []
            for i, segment in enumerate(chain):
                if weight[i] > 0:
                    fy_monomial.append((tuple(segment), weight[i]))
            fy_monomials[degree].add(frozenset(fy_monomial))
    return fy_monomials

def generate_conjugacyClasses(n):
    def subdivide_set(partition_set):
        partitions = []
        elements = list(range(n))
        for length in partition_set:
            partition = elements[:length]
            partitions.append(partition)
            for i in range(length):
                elements.pop(0)
        return partitions

    conjugacy_classes = Partitions(n).list() # type: ignore
    subdivided_partitions = [subdivide_set(part) for part in conjugacy_classes]

    class_actions = []
    for partition in subdivided_partitions:
        irr_representation = {}
        for x in partition:
            for i, element in enumerate(x):
                irr_representation[element] = x[(i + 1) % len(x)]
        class_actions.append(irr_representation)
        
    return conjugacy_classes, class_actions

def mutable_fyMonomials(fy_monomials):
    mutable_fyMonomials = []
    for graded_piece in fy_monomials:
        mutable_graded_piece = [list(monomial) for monomial in graded_piece]
        mutable_fyMonomials.append(mutable_graded_piece)

    return mutable_fyMonomials

def apply_map(monomial, mapping):
    mapped_monomial = []
    for term in monomial:
        new_term = tuple(sorted([mapping[int(x)] for x in term[0]]))
        mapped_monomial.append(tuple((tuple(new_term), term[1])))
    return tuple(sorted(mapped_monomial))

def monomial_equality(monomial, mapped_monomial):
    #sorting the inner tuples, and outer tuples and so on and on
    sorted_monomial = [(sorted(
        [tuple(sorted(m[0]))]), m[1]
                              ) for m in monomial]
    
    sorted_mapped_monomial = [(sorted(
        [tuple(sorted(m[0]))]), m[1]
                              ) for m in mapped_monomial]
    sorted_monomial.sort()
    sorted_mapped_monomial.sort()

    for i in range(len(sorted_monomial)):
        if sorted_monomial[i] != sorted_mapped_monomial[i]:
            return False

    return True

def character(temp_fyMonomials, irr_representation):
    i = 0
    for monomial in temp_fyMonomials:
        mapped_monomial = tuple(sorted(apply_map(monomial, irr_representation)))
        if monomial_equality(monomial, mapped_monomial): i += 1
    return i

def characters_list(fy_monomials, class_actions):
    mutable_fyMonomialsList = mutable_fyMonomials(fy_monomials)
    characters_list = []
    for i in range(len(mutable_fyMonomialsList)):
        characters_list.append(
            [character(mutable_fyMonomialsList[i], mapping) for mapping in class_actions]
        )
    return characters_list

def representations(n):
    representations = []
    partitions = Partitions(n).list() # type: ignore
    for partition in partitions:
        representations.append(
            SymmetricGroupRepresentation(partition).to_character().values() # type: ignore
            )
    return representations

def decomposition(characters_list, n):
    # solving the equation Ax = b.

    trace_matrix = np.matrix(representations(n), dtype = float)
    # print("Character Table : ", trace_matrix)
    decomposition = []
    A = np.array(trace_matrix.transpose())
    for v in characters_list:
        v.reverse()
        rhs_array = np.array(v, dtype=float)
        solution = np.linalg.solve(A, rhs_array)
        decomposition.append(list(np.array(np.round(solution), dtype = int)))

    return decomposition

print(representation_polynomial(9,9))

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 7, 6, 0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [28, 42, 51, 15, 44, 25, 0, 25, 21, 10, 0, 0, 8, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [56, 105, 150, 60, 150, 120, 10, 94, 120, 64, 24, 0, 50, 62, 21, 18, 0, 0, 10, 11, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0], [70, 140, 210, 90, 220, 190, 20, 143, 201, 110, 51, 1, 87, 114, 48, 42, 3, 0, 20, 28, 2, 11, 3, 0, 0, 1, 0, 0, 0, 0], [56, 105, 150, 60, 150, 120, 10, 94, 120, 64, 24, 0, 50, 62, 21, 18, 0, 0, 10, 11, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0], [28, 42, 51, 15, 44, 25, 0, 25, 21, 10, 0, 0, 8, 8, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [8, 7, 6, 0, 4, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
